In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import tensorflow as tf
import pandas as pd
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.DRN.DRN_load_score import *  # Load DRN_scores
from src.models.EMOS_local.EMOS_local_load_score import *  # Load EMOS_local_scores
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models#
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

2023-06-09 16:33:39.040999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-09 16:33:39.041023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 0. Setup

In [3]:
run = 0
path = f"/Data/Delong_BA_Data/scores/DRN_hyper_scores/DRN_hyper_scores_dataframe_5_run_{run}.csv"
df_DRN_hyper_scores_0 = pd.read_csv(path)

<IPython.core.display.Javascript object>

### 1. Functions

In [6]:
def df_split_lead(df):
    return [
        df.loc[df["lead_time"] == 0],
        df.loc[df["lead_time"] == 15],
        df.loc[df["lead_time"] == 30],
    ]

<IPython.core.display.Javascript object>

In [7]:
df_DRN_hyper_split_0 = df_split_lead(df_DRN_hyper_scores_0)

<IPython.core.display.Javascript object>

#### Run 0

In [8]:
df_DRN_hyper_split_0[0].sort_values("score").head()

,lead_time,hidden_layer,emb_size,batch_size,epochs,lr,optimizer,activation,score
8,0,[],5,1024,30,0.005,Adam,relu,0.555016
5,0,[],5,512,30,0.005,Adam,relu,0.555110
17,0,[],10,1024,30,0.005,Adam,relu,0.555141
7,0,[],5,1024,30,0.010,Adam,relu,0.555490
26,0,[],15,1024,30,0.005,Adam,relu,0.555579


<IPython.core.display.Javascript object>

In [9]:
df_DRN_hyper_split_0[1].sort_values("score").head()

,lead_time,hidden_layer,emb_size,batch_size,epochs,lr,optimizer,activation,score
35,15,[],5,1024,30,0.005,Adam,relu,1.100667
44,15,[],10,1024,30,0.005,Adam,relu,1.100702
41,15,[],10,512,30,0.005,Adam,relu,1.100864
53,15,[],15,1024,30,0.005,Adam,relu,1.100892
32,15,[],5,512,30,0.005,Adam,relu,1.100895


<IPython.core.display.Javascript object>

In [10]:
df_DRN_hyper_split_0[2].sort_values("score").head()

,lead_time,hidden_layer,emb_size,batch_size,epochs,lr,optimizer,activation,score
65,30,[],10,256,30,0.005,Adam,relu,1.391657
77,30,[],15,512,30,0.005,Adam,relu,1.391763
71,30,[],10,1024,30,0.005,Adam,relu,1.391784
62,30,[],5,1024,30,0.005,Adam,relu,1.392059
68,30,[],10,512,30,0.005,Adam,relu,1.392120


<IPython.core.display.Javascript object>

In [11]:
df_DRN_hyper_split_0[0]

,lead_time,hidden_layer,emb_size,batch_size,epochs,lr,optimizer,activation,score
0,0,[],5,256,30,0.050,Adam,relu,0.564222
1,0,[],5,256,30,0.010,Adam,relu,0.555739
2,0,[],5,256,30,0.005,Adam,relu,0.556146
3,0,[],5,512,30,0.050,Adam,relu,0.562363
4,0,[],5,512,30,0.010,Adam,relu,0.556169
5,0,[],5,512,30,0.005,Adam,relu,0.555110
6,0,[],5,1024,30,0.050,Adam,relu,0.557797
7,0,[],5,1024,30,0.010,Adam,relu,0.555490
8,0,[],5,1024,30,0.005,Adam,relu,0.555016
9,0,[],10,256,30,0.050,Adam,relu,0.561774


<IPython.core.display.Javascript object>